In [4]:
import boto3
import json
import psycopg2
import base64
from faker import Faker
import uuid
from datetime import datetime, timedelta
import random
import csv
import io

In [5]:
env='dev'
region_name = 'us-east-1'
account_id=183286346090
secret_name = f'ticketing-postgres-credentials-{env}'
bucket_name = f'ticketing-data-{account_id}-{env}'

In [6]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder \
    .appName("ticketing") \
    .config("spark.jars", "postgresql-42.7.5.jar")\
    .enableHiveSupport() \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/02/01 11:38:33 WARN Utils: Your hostname, DESKTOP-BSGB07T. resolves to a loopback address: 127.0.1.1; using 172.26.48.162 instead (on interface eth0)
25/02/01 11:38:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/02/01 11:38:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
# ajusta configurações do spark para carregamento de tabelas
spark.conf.set("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.set("hive.exec.dynamic.partition", "true")
spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict")
spark.conf.set("hive.merge.tezfiles", "true")
spark.conf.set("hive.merge.smallfiles.avgsize", "128000000")
spark.conf.set("hive.merge.size.per.task", "128000000")

In [8]:
spark.sql("show tables in mercado").show(20, False)

+---------+---------------+-----------+
|namespace|tableName      |isTemporary|
+---------+---------------+-----------+
|mercado  |balances       |false      |
|mercado  |bonus          |false      |
|mercado  |cash_flows     |false      |
|mercado  |companies      |false      |
|mercado  |empty_companies|false      |
|mercado  |incomes        |false      |
|mercado  |sales          |false      |
+---------+---------------+-----------+



In [9]:
def get_secret(secret_name):
    """Retrieves the secret from Secrets Manager."""
    
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except Exception as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # The secret can't be decrypted using the provided KMS key.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred inside Secrets Manager.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a bad request.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # The requested secret was not found.
            raise e
    else:
        # Decrypted secrets are available directly.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return json.loads(secret)  # Return secret as a dictionary
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

In [10]:
try:
    secret = get_secret(secret_name)
    db_host = secret["host"]
    db_name = secret["dbname"]
    db_user = secret["username"]
    db_password = secret["password"]    

except Exception as e:
    print(f"Error retrieving secret or connecting to database: {e}")

connection_string = f"jdbc:postgresql://{db_host}:5432/{db_name}"
connection_string

'jdbc:postgresql://ticketing-postgres-db-instance-dev.c99g50znepkh.us-east-1.rds.amazonaws.com:5432/postgres'

In [43]:
# Read the PostgreSQL table into a DataFrame
sales = (
    spark.read
    .format("jdbc")
    .option("url", connection_string)
    .option("dbtable", 'public.sales')
    .option("user", db_user)
    .option("password", db_password)
    .option("driver", "org.postgresql.Driver")
    .load()
)

events = (
    spark.read
    .format("jdbc")
    .option("url", connection_string)
    .option("dbtable", 'public.events')
    .option("user", db_user)
    .option("password", db_password)
    .option("driver", "org.postgresql.Driver")
    .load()
)

resellers = (
    spark.read
    .format("jdbc")
    .option("url", connection_string)
    .option("dbtable", 'public.resellers')
    .option("user", db_user)
    .option("password", db_password)
    .option("driver", "org.postgresql.Driver")
    .load()
)

sales_channels = (
    spark.read
    .format("jdbc")
    .option("url", connection_string)
    .option("dbtable", 'public.sales_channels')
    .option("user", db_user)
    .option("password", db_password)
    .option("driver", "org.postgresql.Driver")
    .load()
)

partnerships = (
    spark.read
    .format("jdbc")
    .option("url", connection_string)
    .option("dbtable", 'public.partnerships')
    .option("user", db_user)
    .option("password", db_password)
    .option("driver", "org.postgresql.Driver")
    .load()
)

customers = (
    spark.read
    .format("jdbc")
    .option("url", connection_string)
    .option("dbtable", 'public.customers')
    .option("user", db_user)
    .option("password", db_password)
    .option("driver", "org.postgresql.Driver")
    .load()
)

organizers = (
    spark.read
    .format("jdbc")
    .option("url", connection_string)
    .option("dbtable", 'public.organizers')
    .option("user", db_user)
    .option("password", db_password)
    .option("driver", "org.postgresql.Driver")
    .load()
)


# Show the DataFrame
# sales_channels.count()

In [44]:
from pyspark.sql import functions as F

sales = (
    sales
    .withColumn(
        "seller_type",
        F.when(F.col("reseller_id").isNull(), F.lit("organizer")).otherwise(F.lit("reseller"))
    )
    .join(
        events.select('event_id', 'event_type'),
        on="event_id",
        how='left'
    )
    .join(
        resellers.withColumnRenamed("location", "reseller_location"),
        on="reseller_id",
        how='left'
    )
    .join(
        sales_channels,
        on="sales_channel_id",
        how='left'
    )
    .join(
        partnerships,
        on=["organizer_id", "reseller_id"],
        how='left'
    )
    .withColumnRenamed("sale_date", "sale_datetime")
    .withColumn("sale_date", F.to_date(F.col("sale_datetime")))
    .join(
        customers,
        on=["customer_id"],
        how='left'
    )
)
sales.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------+-------------------+-----------+----------+---------------+-----------------+------------+--------------------+---------------+----------+----------+----------+-------------------+
|         customer_id|        organizer_id|         reseller_id|    sales_channel_id|            event_id|             sale_id|            venue_id|ticket_quantity|total_amount|      sale_datetime|seller_type|event_type|  reseller_name|reseller_location|channel_name|      partnership_id|commission_rate|start_date|  end_date| sale_date|      customer_name|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------+-------------------+-----------+----------+---------------+-----------------+------------+-----------------

In [13]:
sales.write.mode("overwrite").partitionBy("sale_date").format("parquet").saveAsTable(
    "mercado.sales"
)

In [14]:
glue_db_name = "mercado"
# Retrieve partition information
partitions_df = spark.sql(f"SHOW PARTITIONS {glue_db_name}.sales")

# Extract the latest partition
if partitions_df.count() > 0:
    latest_partition = partitions_df.orderBy("partition", ascending=False).first()[0]
    latest_partition_value = latest_partition.split("=")[-1]    
else:
    latest_partition_value = '0001-01-01'

In [15]:
import pandas as pd
pd.read_csv("sales/DailySales_01072020_b36f0ffb-e39f-41b0-9f6c-43e1ae055edb.csv")

,Transaction ID,Event Name,Number of Purchased Tickets,Total Amount,Sales Channel,Customer First Name,Customer Last Name,Office Location,Created Date
0,1,9,3,27.88,mobile app,David,King,NaN,2020-01-07
1,2,b,1,149.35,mobile app,Nathaniel,Wilson,NaN,2020-01-07
2,3,d,5,111.88,mobile app,Sara,Schneider,NaN,2020-01-07
3,4,6,2,60.25,mobile app,Michael,Nolan,NaN,2020-01-07
4,5,a,1,35.36,office,Wayne,Yoder,New York,2020-01-07
5,6,c,3,174.22,mobile app,Jean,Webster,NaN,2020-01-07
6,7,0,5,78.32,web,Lauren,Benton,NaN,2020-01-07
7,8,6,5,55.74,web,Teresa,Gonzalez,NaN,2020-01-07
8,9,3,1,31.21,web,Joshua,Carroll,NaN,2020-01-07
9,10,f,5,169.58,web,Christopher,Daniel,NaN,2020-01-07


In [17]:
# # Define your target date and bucket path
# target_date = "05032020"
# s3_bucket_path = "file:/home/maxreis86/toptal/screening-ops/Maxuel-reis/sales/"  # adjust as needed
# file_pattern = f"DailySales_{target_date}_*.csv"  # note the wildcard for reseller ID
# full_path = s3_bucket_path + file_pattern

# # Load all CSV files matching the pattern into one DataFrame
# df = spark.read \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .csv(full_path)

# # Optionally, show the DataFrame schema or sample data
# df.printSchema()
# df.show(5)

In [45]:
import pandas as pd
sales_csv = spark.createDataFrame(pd.read_csv("sales/DailySales_01172019_902c603b-16d1-48e3-b8a6-86bbf69bab95.csv"))
sales_csv = sales_csv.withColumn("reseller_id", F.lit("b36f0ffb-e39f-41b0-9f6c-43e1ae055edb"))

In [46]:
sales_csv = (
    sales_csv
    .withColumnRenamed("Transaction ID", "sale_id")
    .withColumnRenamed("Total Amount", "total_amount")
    .withColumnRenamed("Event Name", "event_name")
    .withColumnRenamed("Number of Purchased Tickets", "ticket_quantity")
    .withColumnRenamed("Created Date", "sale_date")    
    .withColumn("customer_name", F.concat(F.col("Customer First Name"), F.lit(" "), F.col("Customer Last Name")))
    .withColumn(
        "channel_name",
        F.when(
            F.col("Sales Channel")=="office", F.lit("on-the-site")
        ).otherwise(
            F.col("Sales Channel")
        )
    )
    .join(
        partnerships,
        on=["reseller_id"],
        how='left'
    )
    .join(
        resellers.withColumnRenamed("location", "reseller_location"),
        on="reseller_id",
        how='left'
    )
    .join(
        organizers,
        on=["organizer_id"],
        how='left'
    )    
    .join(
        events,
        on=["event_name"],
        how='left'
    )
    .drop(
        "Sales Channel",
        "Customer First Name",
        "Customer Last Name",
        "Office Location",
        "event_name",
        "organizer_name"
    )
    .withColumn("seller_type",F.lit("reseller"))
)
sales_csv.show(5, False)

+------------------------------------+------------------------------------+-------+---------------+------------+----------+----------------+------------+------------------------------------+---------------+----------+----------+---------------+-----------------+------------------------------------+----------+------------------------------------+-----------+
|organizer_id                        |reseller_id                         |sale_id|ticket_quantity|total_amount|sale_date |customer_name   |channel_name|partnership_id                      |commission_rate|start_date|end_date  |reseller_name  |reseller_location|event_id                            |event_type|venue_id                            |seller_type|
+------------------------------------+------------------------------------+-------+---------------+------------+----------+----------------+------------+------------------------------------+---------------+----------+----------+---------------+-----------------+------------------

In [60]:
from datetime import datetime

# Original date string in the format YYYY-MM-DD
date_str = '1001-01-01'

# Parse the date string into a datetime object
date_obj = datetime.strptime(date_str, '%Y-%m-%d').strftime('%m%d%Y')

# Format the datetime object into the desired format: %m%d%Y
# formatted_date = date_obj.strftime('%m%d%Y')

print(date_obj)  # Output: 01010001


01011001


In [47]:
sales = (
    sales
    .withColumn("data_source", F.lit("B2B platform"))
    .unionByName(
        sales_csv.withColumn("data_source", F.lit("3rd party")),
        allowMissingColumns=True
    )
)
sales.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------+-------------------+-----------+----------+---------------+-----------------+------------+--------------------+---------------+----------+----------+----------+-------------------+------------+
|         customer_id|        organizer_id|         reseller_id|    sales_channel_id|            event_id|             sale_id|            venue_id|ticket_quantity|total_amount|      sale_datetime|seller_type|event_type|  reseller_name|reseller_location|channel_name|      partnership_id|commission_rate|start_date|  end_date| sale_date|      customer_name| data_source|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------+-------------------+-----------+----------+---------------+-----------------+----

In [39]:
sales.show(500, False)

+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+---------------+------------+-------------------+-----------+----------+--------------------------+-------------------+------------+------------------------------------+---------------+----------+----------+----------+-------------------+------------+-------------------------------------+-----------------------------+
|customer_id                         |organizer_id                        |reseller_id                         |sales_channel_id                    |event_id                            |sale_id                             |venue_id                            |ticket_quantity|total_amount|sale_datetime      |seller_type|event_type|reseller_name             |reseller_location  |channel_name|partnership_id              